# libf0 -  A Python Library for F0-Estimation in Music Recordings

In [ ]:
import numpy as np
import librosa
from scipy.interpolate import interp1d

import IPython.display as ipd
import matplotlib.pyplot as plt

import libf0

In [ ]:
# Plot function
def plot_f0_trajectory(Y_LF, t, f, f0, t_f0, figsize=(8.5, 3.4), xlim=(0, 11.5), ylim=(2000, 6000)):
    """
    Plot a calculated f0 trajectory on the corresponding spectrogram
    
    Parameters
    ----------
    Y_LF : np.ndarray
        log-frequency spectrogram
    t : np.ndarray
        time axis of the spectrogram
    f : np.ndarray
        log-frequency axis of the spectrogram in cents
    f0 : np.ndarray
        f0 trajectory in cents
    t_f0 : np.ndarray
        time points of the f0 trajectory frames
    figsize : tuple
        figure size
    xlim : tuple
        x-limits
    ylim : tuple
        y-limits
    """
    plt.figure(figsize=figsize)

    plt.imshow(Y_LF, cmap='gray_r', aspect='auto', origin='lower', extent=[t[0], t[-1], f[0], f[-1]])
    plt.plot(t_f0, f0, linestyle='', marker='.', markersize=5, color=[192/256, 0, 0])

    plt.xlim(xlim)
    plt.ylim(ylim)

    plt.gca().tick_params(axis='both', which='major', labelsize=10)
    plt.gca().tick_params(axis='both', which='minor', labelsize=10)
    
    plt.xlabel("Time (seconds)", fontsize=12)
    plt.ylabel("Log-Frequency (cents)", fontsize=12)
    
    cbar = plt.colorbar()
    cbar.ax.get_yaxis().labelpad = 15
    cbar.ax.set_ylabel('Log-Magnitude', rotation=270)

    plt.tight_layout()
    plt.show()

In [ ]:
# load demo audio (a throat microphone recording of a soprano singer)
fn_wav = "./data/DCS_LI_QuartetB_Take03_S1_LRX_excerpt.wav"
x, Fs = librosa.load(fn_wav, sr=22050)
ipd.display(ipd.Audio(x, rate=Fs, normalize=True)) # audio playback

# shared parameters
N = 2048 # window size in samples
H = 256 # hop size in samples
zero_pad = 2048 # zero-padding for STFT (only for visualization)
F_min = 55.0 # minimum frequency of interest in Hz
F_max = 1760.0 # maximum frequency of interest in Hz
R = 10 # resolution of F0-estimations in cents

# calculate magnitude spectrogram of input signal for visualization
X = librosa.stft(x, n_fft=N+zero_pad, hop_length=H, win_length=N, window='hann', pad_mode='constant', center=True)
Y = np.abs(X)
F_coef_lin = librosa.fft_frequencies(sr=Fs, n_fft=N+zero_pad)
T_coef = librosa.frames_to_time(np.arange(X.shape[1]), sr=Fs, hop_length=H)

# interpolate magnitude spectrogram to a logarithmic frequency axis 
B = np.floor((1200 / R) * np.log2(F_max / F_min) + 0.5)
F_coef_log_cents = np.arange(0, B) * R 
F_coef_log_hz = 2 ** (F_coef_log_cents / 1200) * F_min
compute_Y_interpol = interp1d(F_coef_lin, Y, kind='cubic', axis=0)
Y_LF = compute_Y_interpol(F_coef_log_hz)
Y_LF[Y_LF < 0] = 0  # discard negative values after interpolation

# use log-magnitude for visualizations
Y_LF = np.log(1 + Y_LF)

plot_f0_trajectory(Y_LF, T_coef, F_coef_log_cents, [], [])

### YIN

For algorithmic details, see:

Alain de Cheveigné and Hideki Kawahara. YIN, a fundamental frequency estimator for speech and music. Journal of the Acoustical Society of America (JASA), 111(4):1917–1930, 2002.

In [ ]:
# YIN parameters
threshold = 0.15

# run YIN algorithm
f0_yin, t_yin, ap_yin = libf0.yin(x, Fs=Fs, N=N, H=H, F_min=F_min, F_max=F_max, threshold=threshold, verbose=True)

# convert trajectory to cent scale
f0_yin_cents = libf0.hz_to_cents(f0_yin, F_min)

# plot the filtered result
plot_f0_trajectory(Y_LF, T_coef, F_coef_log_cents, f0_yin_cents, t_yin)

# sonify the filtered result (left: sonification, right: original audio)
x_son_yin = libf0.sonify_trajectory_with_sinusoid(f0_yin, t_yin, len(x), Fs=Fs)
ipd.display(ipd.Audio(np.vstack((x_son_yin.reshape(1, -1), x.reshape(1, -1))), rate=Fs, normalize=True))

### pYIN

For algorithmic details, see:

Matthias Mauch and Simon Dixon. pYIN: A fundamental frequency estimator using probabilistic threshold distributions. In IEEE International Conference on Acoustics, Speech and Signal Processing (ICASSP), pages 659–663, Florence, Italy, 2014.

In [ ]:
# set parameters
thresholds = np.arange(0.01, 1, 0.01)
R = 10 # bin resolution in cents

# run pYIN algorithm
f0_pyin, t_pyin, conf_pyin = libf0.pyin(x, Fs=Fs, N=N, H=H, F_min=F_min, F_max=F_max, R=R, thresholds=thresholds)

# convert trajectory to cent scale
f0_pyin_cents = libf0.hz_to_cents(f0_pyin, F_min)

# plot the filtered result
plot_f0_trajectory(Y_LF, T_coef, F_coef_log_cents, f0_pyin_cents, t_pyin)

# sonify the filtered result (left: sonification, right: original audio)
x_son_pyin = libf0.sonify_trajectory_with_sinusoid(f0_pyin, t_pyin, len(x), Fs=Fs)
ipd.display(ipd.Audio(np.vstack((x_son_pyin.reshape(1, -1), x.reshape(1, -1))), rate=Fs, normalize=True))

### Salience Algorithm

For algorithmic details, see:

Justin Salamon and Emilia Gómez. Melody extraction from polyphonic music signals using pitch contour characteristics. IEEE Transactions on Audio, Speech, and Language Processing, 20(6): 1759–1770, 2012.

Meinard Müller. Fundamentals of Music Processing – Using Python and Jupyter Notebooks. Springer
Verlag, 2nd edition, 2021. ISBN 978-3-030-69807-2. doi: 10.1007/978-3-030-69808-9.

In [ ]:
# set parameters
num_harm = 10 # number of harmonics for the summation
freq_smoothing = 11 # length of the smoothing filter

# run the salience algorithm
f0_sal, t_sal, conf_sal = libf0.salience(x, Fs=Fs, N=N, H=H, F_min=F_min, F_max=F_max, R=R, num_harm=num_harm, freq_smooth_len=freq_smoothing)

# convert trajectory to cent scale
f0_sal_cents = libf0.hz_to_cents(f0_sal, F_min)

# plot the result
plot_f0_trajectory(Y_LF, T_coef, F_coef_log_cents, f0_sal_cents, t_sal)

# sonify the result (left: sonification, right: original audio)
x_son_sal = libf0.sonify_trajectory_with_sinusoid(f0_sal, t_sal, len(x), Fs=Fs)
ipd.display(ipd.Audio(np.vstack((x_son_sal.reshape(1, -1), x.reshape(1, -1))), rate=Fs, normalize=True))

### SWIPE

For algorithmic details, see:

Arturo Camacho and John G. Harris. A sawtooth waveform inspired pitch estimator for speech and music. The Journal of the Acoustical Society of America, 124(3):1638–1652, 2008.

In [ ]:
# set parameters
threshold = 0.5 # confidence threshold between 0 and 1

# run the SWIPE algorithm
f0_swipe, t_swipe, conf_swipe = libf0.swipe(x, Fs, H, F_min, F_max, strength_threshold=threshold)

# convert trajectory to cent scale
f0_swipe_cents = libf0.hz_to_cents(f0_swipe, F_min)

# plot the result
plot_f0_trajectory(Y_LF, T_coef, F_coef_log_cents, f0_swipe_cents, t_swipe)

# sonify the result (left: sonification, right: original audio)
x_son_swipe = libf0.sonify_trajectory_with_sinusoid(f0_swipe, t_swipe, len(x), Fs=Fs)
ipd.display(ipd.Audio(np.vstack((x_son_swipe.reshape(1, -1), x.reshape(1, -1))), rate=Fs, normalize=True))

### SWIPE (slim)

A more efficient and didactic implementation of the SWIPE algorithm.

In [ ]:
# set parameters
threshold = 0.5 # confidence threshold between 0 and 1

# run the SWIPE algorithm
f0_swipes, t_swipes, conf_swipes = libf0.swipe_slim(x, Fs, H, F_min, F_max, strength_threshold=threshold) # a simplified implementation

# convert trajectory to cent scale
f0_swipes_cents = libf0.hz_to_cents(f0_swipes, F_min)

# plot the result
plot_f0_trajectory(Y_LF, T_coef, F_coef_log_cents, f0_swipes_cents, t_swipe)

# sonify the result (left: sonification, right: original audio)
x_son_swipes = libf0.sonify_trajectory_with_sinusoid(f0_swipes, t_swipes, len(x), Fs=Fs)
ipd.display(ipd.Audio(np.vstack((x_son_swipes.reshape(1, -1), x.reshape(1, -1))), rate=Fs, normalize=True))